На основе регламента контроля и взыскания дебиторской задолженности создайте нейро-ассистента для ответов на вопросы сотрудников по данной базе знаний. Документ с базой знаний можно найти по ссылке: https://docs.google.com/document/d/1UD9lSQy6PcCejUy3k5ARdUBNS-GEhXqOjnMLMZ6r488/edit?usp=sharing

Напишите инструкцию в system самостоятельно. Проверьте работу нейро-ассистента на нескольких самостоятельно сформулированных вопросах.

In [3]:
!pip install -q faiss-cpu==1.8.0 mistralai tiktoken==0.7.0 langchain==0.2.14 langchain_openai==0.1.22 langchain_community==0.2.12 sentence_transformers

In [ ]:
# запустите эту ячейку, если НЕ используете секретный ключ в колабе
import os
import getpass

openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key

OpenAI API Key:··········


In [4]:
# запустите эту ячейку, если используете секретный ключ в колабе
import os
from mistralai import Mistral
from google.colab import userdata
api_key = userdata.get("MISTRAL_API_KEY")
os.environ["MISTRAL_API_KEY"] = api_key

In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re
import requests
from langchain.docstore.document import Document

In [6]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [7]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/1UD9lSQy6PcCejUy3k5ARdUBNS-GEhXqOjnMLMZ6r488/edit?usp=sharing')                 # заполните поле необходимым значением

In [ ]:
data_from_url

'\ufeffРегламент контроля и взыскания дебиторской задолженности\r\nРегламент контроля дебиторской задолженности (далее ДЗ) и взыскания просроченной ДЗ (далее ПДЗ)\r\n1. Назначение Регламента.\r\nНастоящий Регламент предназначен для:\r\n* Системного предупреждения возникновения просроченной дебиторской задолженности (далее – ПДЗ), предотвращения возникновения невозвратной и хронической дебиторской задолженности.\r\n* Сокращения среднего времени просрочек и уменьшения среднего объема ПДЗ.\r\n* Удержания размеров ПДЗ в пределах планово-нормативных значений.\r\n2. Область применения настоящего Регламента.\r\n2.1 Настоящий Регламент применяется ко всем клиентам, работающим с отсрочкой платежей и ко всем случаям возникновения ДЗ и ПДЗ, независимо от категорий клиентов, кроме случаев по которым существует письменное распоряжение руководства о полном или частичном исключении их из области применения настоящего Регламента.\r\n2.2 Регламент вступает в силу с момента возникновения у клиента ДЗ пе

In [20]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))
print("Общее количество чанков: ", len(source_chunks))


Общее количество чанков:  67


In [ ]:
source_chunks

[Document(metadata={'meta': 'data'}, page_content='\ufeffРегламент контроля и взыскания дебиторской задолженности\r\nРегламент контроля дебиторской задолженности (далее ДЗ) и взыскания просроченной ДЗ (далее ПДЗ)\r\n1. Назначение Регламента.\r\nНастоящий Регламент предназначен для:'),
 Document(metadata={'meta': 'data'}, page_content='* Системного предупреждения возникновения просроченной дебиторской задолженности (далее – ПДЗ), предотвращения возникновения невозвратной и хронической дебиторской задолженности.\r\n* Сокращения среднего времени просрочек и уменьшения среднего объема ПДЗ.\r\n* Удержания размеров ПДЗ в пределах планово-нормативных значений.\r\n2. Область применения настоящего Регламента.'),
 Document(metadata={'meta': 'data'}, page_content='2.1 Настоящий Регламент применяется ко всем клиентам, работающим с отсрочкой платежей и ко всем случаям возникновения ДЗ и ПДЗ, независимо от категорий клиентов, кроме случаев по которым существует письменное распоряжение руководства о 

In [21]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Функция для создания эмбеддингов из массива документов
def create_embeddings(documents):
    page_contents = [doc.page_content for doc in documents]
    return model.encode(page_contents)

# Создание эмбеддингов
embeddings = create_embeddings(source_chunks)

# Вывод эмбеддингов
for i, embedding in enumerate(embeddings):
    print(f"Document {i+1} embedding: {embedding}")

Document 1 embedding: [-9.67903137e-02  5.03771245e-01 -3.34264785e-02 -1.79663766e-02
 -1.66910022e-01  3.21125537e-01  8.79188359e-01 -2.69759774e-01
  3.31535071e-01 -4.44596142e-01  3.04278806e-02 -1.37816295e-01
  2.61800557e-01  4.90676984e-02 -1.27686590e-01  2.48664450e-02
 -2.46433809e-01  1.01693201e+00 -8.53419065e-01  2.28642486e-04
 -6.38219640e-02 -3.99253756e-01  2.95066833e-01 -7.78265670e-02
  6.35767132e-02  4.10959184e-01  2.17692509e-01 -2.53756288e-02
 -3.83128941e-01  2.43421152e-01  5.84634066e-01  2.80650049e-01
  3.62441540e-01 -7.03252330e-02 -9.15431753e-02  3.76132250e-01
 -5.60106277e-01  4.93640974e-02  4.55454320e-01  2.60968447e-01
  4.30283517e-01 -1.98579043e-01  1.08217329e-01  3.30447465e-01
  1.46159396e-01 -1.57773256e-01  5.40102363e-01 -2.80337274e-01
  1.20518371e-01 -3.97676051e-01 -3.02593499e-01 -1.35125756e-01
  4.44513679e-01 -7.27048755e-01 -2.01766551e-01 -1.03592932e+00
  5.39308071e-01  4.43404943e-01 -1.77737504e-01  4.20580432e-02
  9

In [22]:
import numpy as np
import faiss
# Преобразование эмбеддингов в массив numpy
embeddings = np.array(embeddings).astype('float32')

# Создание индекса FAISS
dimension = embeddings.shape[1]  # Размерность векторов
index = faiss.IndexFlatL2(dimension)  # Использование L2 метрики

# Добавление векторов в индекс
index.add(embeddings)

In [28]:
def answer_index(system, topic, query_vector, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    distances, indices =  search_index.search(query_vector, k=4)
    #indices_list = indices[0].tolist()
    # Вывод ближайших документов
    docs = [source_chunks[i] for i in indices[0]]

    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = Mistral(api_key=api_key)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос сотрудника компании. Не упоминай документ с информацией для ответа сотруднику в ответе. Документ с информацией для ответа сотруднику: {message_content}\n\nВопрос сотрудника: \n{topic}"}
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.complete(model="mistral-small-latest", messages=messages)
    answer = completion.choices[0].message.content
    return answer  # возвращает ответ

In [29]:
# проверьте работу нейро-консультанта на нескольких самостоятельно-сформулированных вопросах
topic= "Что будет при просрочке дебиторской задолжности?"
system="Ты консультант по управлению дебиторской задолженностью, ответь на вопрос клиента на основе документа с информацией. Не придумывай ничего от себя, отвечай максимально по документу. Не упоминай Документ с информацией для ответа клиенту. Клиент ничего не должен знать про Документ с информацией для ответа клиенту"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

v = answer_index(system, topic, query_vector, index)
v



 ===========================================: 
message_content :
 
Отрывок документа №1
=====================* промежуточного контроля, в течение действия отсрочки платежа, по договору
* инкассации денежных средств  
Отрывок документа №2
=====================безопасности, взаимодействия, контроля  и мотивации, а также с политикой цен и скидок. Если в силу разных причин это  невозможно, то следует предварительно разделить  клиентскую базу  по категориям риска невозврата товарного кредита и объема возможных потерь в случае невозможности взыскания просроченной дебиторской задолженности. Объем возможных потерь в случае невозможности взыскания во многом зависит от места нахождения клиента в канале сбыта: сети (глобальные, федеральные, региональные) опт, розница.  
Отрывок документа №3
=====================Следует ещё раз отметить, что  указанные изменения в организации должны носить периодический характер. Необходимость и периодичность изменений будет определяться результатами анализа:
* о

'В случае просрочки дебиторской задолженности необходимо предпринять меры для ее взыскания. Важно учитывать различные факторы, такие как уровень риска невозврата и объем возможных потерь, которые могут зависеть от категории клиента и его места в канале сбыта. Регулярный анализ основных показателей дебиторской задолженности, таких как обращаемость и возрастная структура, поможет определить необходимость и периодичность изменений в организации работы с дебиторской задолженностью. Это также способствует эффективному использованию денежных средств предприятия, сохранению клиентов и снижению риска безвозвратной потери.'

In [31]:
topic= "Какие меры можно предпринять для улучшения управления дебиторской задолженностью в компании?"
system="Ты консультант по управлению дебиторской задолженностью, ответь на вопрос клиента на основе документа с информацией. Не придумывай ничего от себя, отвечай максимально по документу. Не упоминай Документ с информацией для ответа клиенту. Клиент ничего не должен знать про Документ с информацией для ответа клиенту"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

v = answer_index(system, topic, query_vector, index)

v


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================* промежуточного контроля, в течение действия отсрочки платежа, по договору
* инкассации денежных средств  
Отрывок документа №2
=====================безопасности, взаимодействия, контроля  и мотивации, а также с политикой цен и скидок. Если в силу разных причин это  невозможно, то следует предварительно разделить  клиентскую базу  по категориям риска невозврата товарного кредита и объема возможных потерь в случае невозможности взыскания просроченной дебиторской задолженности. Объем возможных потерь в случае невозможности взыскания во многом зависит от места нахождения клиента в канале сбыта: сети (глобальные, федеральные, региональные) опт, розница.  
Отрывок документа №3
=====================Поведение Клиента, состояние торговой точки указывает на высокий риск потери денежных средств (далее ДС). 
	Немедленно информирует непосредственного руководителя о сложившейся ситуации. С

'Для улучшения управления дебиторской задолженностью в компании можно предпринять следующие меры:\n\n1. **Проведение промежуточного контроля** в течение действия отсрочки платежа по договору.\n2. **Организация инкассации денежных средств** для обеспечения своевременного получения платежей.\n3. **Интеграция политики безопасности, взаимодействия, контроля и мотивации** с политикой цен и скидок. Если это невозможно, разделите клиентскую базу по категориям риска невозврата и объема возможных потерь.\n4. **Анализ поведения клиента и состояния торговой точки** для оценки риска потери денежных средств. В случае высокого риска немедленно информируйте непосредственного руководителя и согласуйте дальнейшие действия.\n5. **Общение в день платежа** с клиентом, указывая оплату по текущей накладной как одну из целей взаимодействия.\n6. **Создание акта сверки, изменения/дополнения к договору поставки** с условиями реструктуризации и получение письменно оформленной просьбы об отсрочке платежа с указан

In [32]:
topic= "Какие есть варианты взыскания дебеторской задолжности?"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')
v = answer_index(system, topic, query_vector, index)

v


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Важно, что таблица старения дебиторской задолженности в программе 1С настроена таким же образом. Следовательно, руководитель или владелец бизнеса, отслеживая изменение сумм в указанных временных интервалах, может наблюдать эффективность работы подразделений и отдельных сотрудников компании, а также определять объём их текущей загрузки. Временные периоды старения дебиторской задолженности необходимо изменять в зависимости от отраслевых показателей или в связи с изменениями внешнеэкономической ситуации. На практике, временные периоды контроля, в сочетании с регламентами контроля, должны  
Отрывок документа №2
=====================безопасности, взаимодействия, контроля  и мотивации, а также с политикой цен и скидок. Если в силу разных причин это  невозможно, то следует предварительно разделить  клиентскую базу  по категориям риска невозврата товарного кредита и объема возможных по

'Варианты взыскания дебиторской задолженности включают в себя несколько шагов и методов:\n\n1. **Проведение переговоров**: Начальной стадией может быть прямой контакт с клиентом для обсуждения возможностей оплаты. Это может включать уточнение времени и даты контакта, а также благодарность за понимание важности своевременной оплаты.\n\n2. **Напоминания и уведомления**: Использование маршрутного листа для уточнения времени и даты контакта. В случае сомнений в своевременной оплате, можно делать пометки о необходимости заранее напомнить клиенту о платеже.\n\n3. **Взаимодействие с руководством**: В случае необходимости можно запрашивать содействие супервайзера для достижения цели.\n\nЭти шаги помогут вам эффективно управлять дебиторской задолженностью и минимизировать риски невозврата.'

In [33]:
topic= "Какие есть варианты взыскания дебеторской задолжности если клиент не идет на контакт?"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')
v = answer_index(system, topic, query_vector, index)

v


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================Уточняет время и дату контакта  (на основе  маршрутного листа). Благодарит Клиента за конструктивный подход, прощается.
	b.
	Поведение Клиента, состояние торговой точки вызывает сомнения в своевременной оплате.
	Уточняет время и дату контакт  (на основе  маршрутного листа). Благодарит Клиента за понимание того, что своевременная оплата важный элемент рабочих  взаимоотношений,  прощается с Клиентом.
        В маршрутном листе делает пометку о необходимости заранее напомнить Клиенту о платеже. При необходимости запрашивает содействие супервайзера (далее СВ) в достижении цели.
	c.  
Отрывок документа №2
=====================безопасности, взаимодействия, контроля  и мотивации, а также с политикой цен и скидок. Если в силу разных причин это  невозможно, то следует предварительно разделить  клиентскую базу  по категориям риска невозврата товарного кредита и объема возможных потерь в 

'В случае, если клиент не идет на контакт, важно предпринять несколько шагов для взыскания дебиторской задолженности. Во-первых, можно воспользоваться автоматизированными системами напоминаний, которые будут регулярно напоминать клиенту о необходимости оплаты. Во-вторых, можно провести анализ клиентской базы и разделить клиентов по категориям риска невозврата, чтобы сосредоточиться на тех, кто наиболее вероятно приведет к значительным потерям. В-третьих, можно использовать внешние коллекторские агентства или юридические службы для взыскания задолженности. Важно также пересмотреть политику цен и скидок, чтобы минимизировать риски в будущем.'

In [34]:
topic= "Какие есть административные, юридические и банковские инструменты для взыскания задолжности?"
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')
v = answer_index(system, topic, query_vector, index)

v


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================безопасности, взаимодействия, контроля  и мотивации, а также с политикой цен и скидок. Если в силу разных причин это  невозможно, то следует предварительно разделить  клиентскую базу  по категориям риска невозврата товарного кредита и объема возможных потерь в случае невозможности взыскания просроченной дебиторской задолженности. Объем возможных потерь в случае невозможности взыскания во многом зависит от места нахождения клиента в канале сбыта: сети (глобальные, федеральные, региональные) опт, розница.  
Отрывок документа №2
=====================* промежуточного контроля, в течение действия отсрочки платежа, по договору
* инкассации денежных средств  
Отрывок документа №3
=====================Важно, что таблица старения дебиторской задолженности в программе 1С настроена таким же образом. Следовательно, руководитель или владелец бизнеса, отслеживая изменение сумм в указанных вр

'Для взыскания задолжности можно использовать различные административные, юридические и банковские инструменты. Вот некоторые из них:\n\n1. **Административные инструменты:**\n   - Внесение задолжника в специальные реестры, такие как реестр задолжников по налогам и сборам.\n   - Использование административных штрафов и санкций.\n   - Введение предупреждений и уведомлений о необходимости оплаты.\n\n2. **Юридические инструменты:**\n   - Судебное взыскание через суд.\n   - Использование арбитражных судов для разрешения споров.\n   - Введение имущественного исполнения по решению суда.\n\n3. **Банковские инструменты:**\n   - Обращение в банк для блокировки счетов задолжника.\n   - Использование услуг коллекторских агентств, которые могут работать через банковские системы.\n   - Введение международных санкций и блокировок через банковские системы, если речь идет о зарубежных клиентах.\n\nЭти меры помогут вам эффективно справляться с проблемой невозврата дебиторской задолженности.'